In [2]:
from ultralytics import YOLO
import cv2
import cvzone
import math

In [3]:
from sort import *

In [4]:
model = YOLO('.../yolov8n.pt')
classNames = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
              'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 
              'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 
              'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 
              'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 
              'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 
              'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 
              'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 
              'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 
              'teddy bear', 'hair drier', 'toothbrush']


In [31]:
#create a webcam object
cap = cv2.VideoCapture('../Videos/cars2.mp4')
#setting the width of videocapture
cap.set(4, 480)

#Tracking
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limits = [200,800,4500, 800]
totalCount = []

while True:
    #read a frame from the webcam
    #if the frame is successfully read, cap.read() returns boolean true and returns a numpy array of pixel values that represent the image captured
    success, img = cap.read()
    
    #imgGraphics = cv2.imread('graphics.png', cv2.IMREAD_UNCHANGED)
    #cvzone.overlayPNG(img, imgGraphics, (0,0))
    
    #we are passing the image into the Yolo model and in streaming mode allowing it to use generators for better efficiency and this returns a generator
    #a generator is an iterable like lists and tuples but the difference is that it uses lazy evaluation i.e the values are produced on the fly only when needed
    #when lists are defined, the values are immediately computed and stored in memory
    #generators are so memory efficient coz they use produce one element at a time making them useful for large datasets
    results = model(img, stream=True)
    
    #create an array of detections
    detections = np.empty((0, 5))
    
    #lets iterate over them
    #r is an object that has the following attributes: boxes, keypoints, masks, names, orginal image etc
    for r in results:
        boxes=r.boxes
        for box in boxes:
            #Bounding boxes are defined either x_min,y_min,x2_max,y2_max  x_center,y_center,w,h coordinate formats
            x1,y1,x2,y2 = box.xyxy[0]
            
            #convert the coordinates from tensor type to int type
            x1,x2,y1,y2 =int(x1),int(x2),int(y1),int(y2)
            
            
            print(x1,y1,x2, y2)
            
            #draw the rectange around the frame or image
            #cv2.rectangle(img, (x1,y1), (x2,y2), (255, 0, 255), 3)
            
            #we can use the cvzone for drawing rectange as well. Call the cornerRect method and input img and the x1,y1,w,h
            bbox = x1,y1, x2-x1, y2-y1
            
            #Confidence value that is in the tensor type and we round it up to the ceil and express in 2 dp
            conf = math.ceil((box.conf[0]*100))/100
            
            cls = int(box.cls[0])
           
            
            #We want to detect cars only if they have a confidence level > 0.25
            currentClass = classNames[cls]
            if currentClass == "car" or currentClass == "bus" or currentClass == "truck" or currentClass == "motorbike" and conf > 0.25:
                #put the classname and label
                #cvzone.putTextRect(img, "{} {}".format(classNames[cls], conf), (max(0,x1), max(35,y1)), scale=1.6)
                
                #put a box only around if its part of what we want
                #cvzone.cornerRect(img, bbox, l=30, rt=5)
                
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))


    resultsTracker = tracker.update(detections)
    
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0,0,255),5)
    for result in resultsTracker:
        x1, y1, x2, y2, Id = result
        x1,x2,y1,y2 =int(x1),int(x2),int(y1),int(y2)

        print(result)
        cvzone.cornerRect(img, bbox, l=9, rt=2, colorR=(255,0,255) )
        cvzone.putTextRect(img, "{}".format(int(Id)), (max(0,x1), max(35,y1)), scale=2, offset = 10, thickness=3)

        cx,cy = x1+(x2-x1)//2, y1+(y2-y1)//2
        cv2.circle(img, (cx,cy), 5, (255,0,255), cv2.FILLED)
        
        if limits[0] < cx <limits[2] and limits[1]-20 <cy <limits[3]+20:
            if totalCount.count(Id) == 0:
                totalCount.append(Id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0,255,0),5)

   
    cvzone.putTextRect(img, "Count: {}".format(len(totalCount)), (50, 50), scale=2, offset = 10, thickness=3)

    #cv2.putText(img, str(len(totalCount)), (255,100), cv2.FONT_HERSHEY_PLAIN, 5, (50,50,255), 8)
    #window name
    window_name = "Image"
    
    #display the frame img in a new window labeled 'Image'
    
    cv2.imshow(window_name, img)
    
    #move the window to my primary monitor
    cv2.moveWindow(window_name, 0, 0)
     
    
    cv2.waitKey(2)  


2639 0 3033 207
1788 180 2274 633
1784 180 2274 634
0: 384x640 2 cars, 1 truck, 73.5ms
Speed: 3.2ms preprocess, 73.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[       1784         180        2274         634         307]
[       1788         180        2274         633         306]
[       2639           0        3033         207         305]

2640 2 3034 215
1775 192 2267 650
1777 191 2267 648
0: 384x640 2 cars, 1 truck, 80.2ms
Speed: 2.7ms preprocess, 80.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1776.6      191.36      2267.4      647.64         307]
[     1775.1      191.93      2266.9      650.07         306]
[     2638.6      2.7285      3035.4      214.27         305]

2642 1 3042 222
1771 198 2265 663
1773 200 2265 663
0: 384x640 2 cars, 1 truck, 70.6ms
Speed: 2.2ms preprocess, 70.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1772.1      201.09      2265.5      661.97         307]
[     1769.

Speed: 2.2ms preprocess, 82.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[       1878      7.0721      2268.2      134.63         309]
[     2664.5      10.918      3189.3      505.72         308]
[     1419.8      572.42      2086.7      1184.6         306]

1883 0 2262 145
1382 591 2069 1221
2678 6 3183 523
2676 3 3179 526
0: 384x640 3 cars, 1 truck, 78.0ms
Speed: 2.3ms preprocess, 78.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1873.8      5.6902      2271.5      141.11         309]
[     2667.6      12.089      3192.3      517.61         308]
[     1392.2      592.56      2069.3      1213.9         306]

1367 618 2056 1244
1884 0 2264 160
2674 3 3188 538
2667 1 3183 541
0: 384x640 3 cars, 1 truck, 78.4ms
Speed: 2.2ms preprocess, 78.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[     1871.3      6.6188      2275.7      151.89         309]
[     2667.4      11.474      3195.3      530.51         308]
[     1

Speed: 2.2ms preprocess, 98.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[       1816      6.9997      2290.8      328.56         311]
[     2608.2      184.65      3222.2      867.34         310]
[     1818.7      10.473      2287.3      323.82         309]
[     2610.1      184.38      3226.4       865.4         308]
[     778.73      1078.7      1672.4      1891.3         306]

708 1109 1642 1954
2583 199 3232 894
1808 0 2273 350
1809 0 2277 352
2556 205 3230 892
1231 1307 1385 1440
0: 384x640 1 person, 3 cars, 2 trucks, 79.1ms
Speed: 2.9ms preprocess, 79.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1803.7      7.1553      2284.3      341.73         311]
[     2594.8      198.79      3224.4      891.63         310]
[     1808.4      10.915      2283.6      338.53         309]
[     2582.9      201.18      3220.2      889.68         308]
[     727.66      1113.2        1638      1939.8         306]

655 1142 1607 2008
2571 224 324

Speed: 4.1ms preprocess, 80.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[     1627.4      85.748      2203.9      669.96         311]
[     2310.1      607.64      3144.3        1493         310]

1618 104 2194 697
1619 104 2191 694
2258 631 3141 1545
2758 877 2932 996
2255 631 3142 1545
0: 384x640 1 person, 2 cars, 2 trucks, 85.9ms
Speed: 3.0ms preprocess, 85.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[     1616.2      100.71      2197.8       692.6         311]
[     2278.8      634.29      3132.7      1534.4         310]

2253 661 3126 1582
1604 125 2186 724
2748 923 2897 1037
1603 127 2187 726
0: 384x640 1 person, 1 car, 1 bus, 1 truck, 85.5ms
Speed: 2.3ms preprocess, 85.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[     1603.1      120.86      2190.5      719.93         311]
[     2258.9      662.16        3124      1574.3         310]
[     1602.6      124.18      2190.4      713.44         309]

2211 695 

[     1117.3      571.52      1913.3      1375.3         311]
[     1406.5      1591.3      2646.2      2213.4         308]

1375 1733 2559 2145
1063 599 1886 1430
1063 597 1886 1431
1379 1731 2564 2148
0: 384x640 2 cars, 2 trucks, 78.5ms
Speed: 2.8ms preprocess, 78.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[       1078      600.07      1886.4      1417.2         311]
[     1361.7      1660.1      2600.2      2212.7         310]
[       1354      1657.4      2600.4      2216.1         308]

1020 615 1858 1469
1324 1807 2495 2143
0: 384x640 1 car, 1 truck, 71.2ms
Speed: 2.1ms preprocess, 71.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1036.3      623.29      1857.9      1455.6         311]
[     1299.2      1733.3      2547.3      2210.9         310]

990 650 1832 1512
1291 1883 2463 2145
0: 384x640 1 car, 1 truck, 73.1ms
Speed: 2.2ms preprocess, 73.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1000.1 

...

219 260 1011 928
2779 3 3597 626
2034 2 2384 102
0: 384x640 3 cars, 73.3ms
Speed: 2.5ms preprocess, 73.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[       2757      29.197      3618.4      593.62         329]
[      220.3      251.83      1022.3      923.54         327]

2758 6 3587 659
182 287 989 964
2932 68 3088 200
2011 2 2344 119
2008 1 2646 122
0: 384x640 1 person, 4 cars, 72.2ms
Speed: 2.5ms preprocess, 72.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     2737.5      31.101      3610.1       630.1         329]
[     182.63       280.4      994.99      961.09         327]

141 324 959 1009
2728 4 3573 698
2907 91 3072 232
1990 2 2637 152
2164 1 2633 134
0: 384x640 1 person, 4 cars, 74.4ms
Speed: 2.6ms preprocess, 74.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     2712.1      31.669      3596.9      667.25         329]
[     142.79      314.64      965.52      1003.9         327]

99 357 929 1039
2711 5 3

Speed: 2.3ms preprocess, 75.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     2086.8      673.76      3229.6      1737.5         335]
[       1435      16.038      2398.9      649.94         334]
[    -73.346      1143.6       228.4      1970.1         327]

1413 8 2360 691
2029 726 3208 1826
2295 949 2535 1148
2029 726 3205 1823
0 1327 90 1935
0: 384x640 1 person, 3 cars, 1 truck, 75.1ms
Speed: 2.4ms preprocess, 75.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
[     2032.4      727.39      3203.4      1818.1         335]
[     1396.9      18.924      2377.5       680.4         334]

1370 3 2338 732
2228 1010 2456 1218
1974 785 3174 1908
1971 784 3177 1906
0: 384x640 1 person, 2 cars, 1 truck, 68.6ms
Speed: 2.3ms preprocess, 68.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[     1976.8      785.31      3174.2      1902.2         335]
[     1354.6      19.359      2356.2      715.16         334]

1928 842 3153 1994
1

Speed: 2.9ms preprocess, 74.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[    -76.265       567.5      1350.1        2152         334]

1 635 1229 2138
1882 2 2586 193
0: 384x640 1 car, 1 truck, 74.8ms
Speed: 2.2ms preprocess, 74.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

2 690 1154 2148
1850 2 2555 221
0: 384x640 1 car, 1 truck, 67.3ms
Speed: 4.8ms preprocess, 67.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[    -60.256       682.3      1206.9      2162.7         331]

1 745 1088 2144
1819 5 2551 247
0: 384x640 1 car, 1 truck, 67.7ms
Speed: 2.8ms preprocess, 67.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[     1794.8      11.595      2573.5      240.25         339]
[    -70.808      718.65      1147.8      2183.6         331]

0 803 1019 2135
1784 2 2527 274
0: 384x640 1 car, 1 truck, 70.2ms
Speed: 2.3ms preprocess, 70.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


0 740 1294 1963
708 1007 934 1209
0: 384x640 1 person, 1 car, 70.9ms
Speed: 2.6ms preprocess, 70.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[    0.84449      747.63      1306.8      1933.3         339]

0 823 1228 2033
618 1071 855 1297
0: 384x640 1 person, 1 car, 72.6ms
Speed: 2.5ms preprocess, 72.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[    -41.065      814.35      1255.8      2017.2         339]

1 885 1152 2112
535 1139 775 1378
0: 384x640 1 person, 1 car, 72.5ms
Speed: 2.6ms preprocess, 72.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[    -66.931      879.18      1201.2      2098.2         339]

442 1224 679 1453
0 953 1073 2148
0 953 1076 2145
0: 384x640 1 person, 1 car, 1 truck, 72.0ms
Speed: 2.5ms preprocess, 72.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[    -84.096      936.76      1141.3      2158.4         339]

0 1020 993 2145
340 1303 589 1546
0 1018 994 2141
0: 384x6

error: OpenCV(4.7.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
